In [1]:
import torch
import tensorflow as tf
import numpy as np
import xlns as xl

In [2]:
def encode_lns_torch(values: torch.Tensor) -> torch.Tensor:
    values = values.to(torch.float64)

    abs_vals = torch.abs(values)

    log_vals = torch.log2(abs_vals)
    scaled = log_vals * (2**23)
    rounded = torch.round(scaled)
    encoded = (rounded.to(torch.int64) * 2)
    sign_bit = (values < 0).to(torch.int64)
    encoded = encoded | sign_bit

    return encoded


def lns_add_pytorch(x, y, F=23):
    
    

    max_val = torch.maximum(x, y)
    
    diff = -torch.abs(x // 2 - y // 2)

    
    sign_bit = (x ^ y) & 1
    
    sb_result = sbdb_ufunc_ideal_pytorch(diff, sign_bit)
    
    func_output = max_val + sb_result

    final_result = func_output

    return final_result


def sbdb_ufunc_ideal_pytorch(z, s, F=23):
    z_float = z.to(torch.float64) / (2**F)
    s_float = s.to(torch.float64)
    
    min_val = torch.minimum(-s_float, z_float)

    pow_result = torch.pow(2.0, min_val)
    
    inner_term = 1.0 - 2.0 * s_float + pow_result
    
    log_result = torch.log2(torch.abs(inner_term))

    result = torch.round(log_result * (2**F)).to(torch.int64)
    result = (result * 2)
    
    return result



In [3]:
(xl.xlnsnp([10,3]) + xl.xlnsnp([1,7])).nd

array([58039632, 55732706])

In [4]:
a = torch.tensor([10,3])
b = torch.tensor([1,7])

a_1 = encode_lns_torch(a)
b_1 = encode_lns_torch(b)
lns_add_pytorch(a_1,b_1)

tensor([58039632, 55732706])

In [5]:
def encode_lns_tf(values):
    values = tf.cast(values, tf.float64)
    abs_vals = tf.abs(values)
    log_vals = tf.math.log(abs_vals) / tf.math.log(tf.cast(2, tf.float64))
    scaled = log_vals * (2**23)
    rounded = tf.round(scaled)
    encoded = tf.cast(rounded, tf.int64) * 2
    sign_bit = tf.cast(values < 0, tf.int64)
    encoded = encoded | sign_bit

    return encoded

def sbdb_ufunc_ideal_tf(z, s, F=23):
    z_float = tf.cast(z, tf.float64) / (2**F)
    s_float = tf.cast(s, tf.float64)
    min_val = tf.minimum(-s_float, z_float)
    pow_result = tf.pow(tf.constant(2.0, dtype=tf.float64), min_val)
    inner_term = 1.0 - 2.0 * s_float + pow_result
    abs_inner = tf.abs(inner_term)
    log_result = tf.math.log(abs_inner) / tf.math.log(tf.constant(2.0, dtype=tf.float64))
    scaled_result = log_result * (2**F)
    rounded_result = tf.round(scaled_result)
    # is_negative = tf.cast(inner_term < 0, tf.int64)
    result = tf.cast(rounded_result, tf.int64) * 2
    
    return result

def lns_add_tensorflow(x, y, F=23):

    max_val = tf.maximum(x, y)
    diff = -tf.abs(x // 2 - y // 2)
    sign_bit = (x ^ y) & 1
    sb_result = sbdb_ufunc_ideal_tf(diff, sign_bit, F)
    func_output = max_val + sb_result
    
    return func_output


In [6]:
x = tf.constant([2,3], dtype=tf.int64)
y = tf.constant([1,-1], dtype=tf.int64)

x_1 = encode_lns_tf(x)
y_1 = encode_lns_tf(y)
lns_add_tensorflow(x_1, y_1)

<tf.Tensor: shape=(2,), dtype=int64, numpy=array([26591258, 16777216])>